In [95]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import re
import joblib
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, get_scorer_names
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from xgboost import XGBRegressor
from copy import deepcopy
warnings.filterwarnings("ignore")

## 데이터 전처리

In [121]:
df = pd.read_csv("./data/merge_all/merge_drop.csv")
# df.head()

In [122]:
move_avr = [5, 6, 20, 24, 60, 72, 300]
for avr in move_avr : 
    df["이동평균가_"+str(avr)+"일"] = df["평균"].rolling(avr).mean()

In [123]:
df["거래년월일"] = pd.to_datetime(df["거래년월일"])
df.set_index("거래년월일", inplace=True)

In [124]:
df.dropna(how="any", inplace=True)
# df.head()

In [125]:
df = df[['자동차용 경유 가격 (원)', '전월비(%)', '거래량', '이동평균가_5일', '이동평균가_6일',
       '이동평균가_20일', '이동평균가_24일', '이동평균가_60일', '이동평균가_72일', '이동평균가_300일', '평균']]

In [126]:
df.rename(columns={"평균":"배추가격"},inplace=True)
# df.info()

In [127]:
# df.columns

In [128]:
feature = df.drop(columns=["거래량", "배추가격", "이동평균가_6일", "이동평균가_24일", "이동평균가_72일"])
target = df["배추가격"]

In [104]:
## StandardScaler를 이용한 정규화

In [129]:
std_feat = StandardScaler()
std_tar = StandardScaler()

std_feat.fit(feature)
std_tar.fit(target.values.reshape(-1,1))

X = std_feat.transform(feature)
y = std_tar.transform(target.values.reshape(-1,1))

In [106]:
## trainset과 testset을 분리

In [130]:
len_x = int(len(X)*0.8)
len_y = int(len(y)*0.8)

X_train, X_test = X[:len_x], X[len_x:]
y_train, y_test = y[:len_y], y[len_y:]

## 12월 테스트용 데이터 전처리

In [131]:
## 12월 테스트용 데이터 프레임 만들기
df_price = pd.read_csv("./data/price/price_baechoo.csv")
df_price.set_index("구분", inplace=True)   # 구분 column을 인덱스로 설정
df_price.rename(columns={"평균" : "배추가격"})
df_price = df_price[df_price.index >= "2022-12-01"]
df_price.index.name = ""
df_price.index = pd.to_datetime(df_price.index)


In [132]:
local_path = "C:/workspace/datas/"
df_oil = pd.read_csv(local_path+"2022년_12월_주유소_경유가격.csv", encoding="cp949")
df_oil["구분"] = df_oil["구분"].apply(lambda x : datetime.strptime(str(x), "%Y년%m월%d일"))
df_oil.set_index("구분", inplace=True)
df_oil.index.name = ""
# df_oil

In [133]:
df_supply = pd.read_csv("./data/transaction_supply/transaction_supply_baechoo.csv")
df_supply["거래년월일"] = pd.to_datetime(df_supply["거래년월일"])
df_supply.set_index("거래년월일", inplace=True)
df_supply = df_supply[df_supply.index >= "2022-12-01"]
df_supply.index.name = ""
# df_supply

In [134]:
df_test = pd.concat([df_oil, df_price, df_supply], axis=1)
# df_test = df_test.join(df_supply, how="inner")
df_test.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 22 entries, 2022-12-01 to 2022-12-22
Freq: D
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자동차용경유  22 non-null     float64
 1   평균      13 non-null     float64
 2   거래량     17 non-null     float64
dtypes: float64(3)
memory usage: 704.0 bytes


In [135]:
df_test.dropna(how="any",inplace=True)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 13 entries, 2022-12-01 to 2022-12-20
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자동차용경유  13 non-null     float64
 1   평균      13 non-null     float64
 2   거래량     13 non-null     float64
dtypes: float64(3)
memory usage: 416.0 bytes


In [136]:
df_test.columns = ['자동차용 경유 가격 (원)', '배추가격', '거래량' ]
# df_test.info()

In [137]:
df_merge_pred = pd.concat([df, df_test])
df_merge_pred.tail(20)

,자동차용 경유 가격 (원),전월비(%),거래량,이동평균가_5일,이동평균가_6일,이동평균가_20일,이동평균가_24일,이동평균가_60일,이동평균가_72일,이동평균가_300일,배추가격
2022-11-22,1879.15,-8.0,187060.0,6582.0,6599.000000,7114.350,7364.500000,17866.383333,17752.263889,11956.331667,6260.0
2022-11-23,1879.15,-8.0,909340.0,6422.0,6501.666667,7015.750,7241.875000,17688.383333,17573.930556,11945.515000,6100.0
2022-11-24,1879.15,-8.0,714993.0,6355.6,6368.333333,6908.150,7138.416667,17505.050000,17396.708333,11937.081667,6100.0
2022-11-25,1879.15,-8.0,1079776.0,6172.0,6313.000000,6817.350,7031.625000,17234.383333,17199.763889,11924.265000,6100.0
2022-11-28,1879.15,-8.0,201478.0,6044.8,6087.333333,6710.925,6927.125000,16932.116667,17006.763889,11910.411667,5664.0
2022-11-29,1879.15,-8.0,77130.0,5911.2,5969.333333,6623.025,6823.791667,16611.983333,16838.875000,11903.151667,5592.0
2022-11-30,1879.15,-8.0,119783.0,5739.6,5799.666667,6543.350,6698.375000,16217.683333,16704.736111,11890.411667,5242.0
2022-12-01,1857.47,NaN,404610.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5462.0
2022-12-05,1850.98,NaN,135374.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5982.0
2022-12-06,1847.19,NaN,83130.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5502.0


In [146]:
df_merge_pred.iloc[-15:-12]

,자동차용 경유 가격 (원),전월비(%),거래량,이동평균가_5일,이동평균가_6일,이동평균가_20일,이동평균가_24일,이동평균가_60일,이동평균가_72일,이동평균가_300일,배추가격
2022-11-29,1879.15,-8.0,77130.0,5911.2,5969.333333,6623.025,6823.791667,16611.983333,16838.875000,11903.151667,5592.0
2022-11-30,1879.15,-8.0,119783.0,5739.6,5799.666667,6543.350,6698.375000,16217.683333,16704.736111,11890.411667,5242.0
2022-12-01,1857.47,NaN,404610.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5462.0


In [154]:
df_merge_pred.iloc[-13]

자동차용 경유 가격 (원)      1857.470000
전월비(%)                      NaN
거래량               404610.000000
이동평균가_5일            5612.000000
이동평균가_6일            5693.333333
이동평균가_20일           6459.350000
이동평균가_24일           6596.125000
이동평균가_60일          15821.716667
이동평균가_72일          16556.152778
이동평균가_300일         11879.271667
배추가격                5462.000000
Name: 2022-12-01 00:00:00, dtype: float64

In [165]:
num_null = len(df_merge_pred[df_merge_pred["이동평균가_5일"].isnull()])

In [166]:
for k in range(len(move_avr)) : 
    for i in range(num_null) :           
        df_merge_pred.iloc[(i-num_null), (k+3)] = df_merge_pred.iloc[(i-num_null-(move_avr[k]-1)):(i-num_null+1),-1].mean()

In [167]:
df_merge_pred.tail(20)

,자동차용 경유 가격 (원),전월비(%),거래량,이동평균가_5일,이동평균가_6일,이동평균가_20일,이동평균가_24일,이동평균가_60일,이동평균가_72일,이동평균가_300일,배추가격
2022-11-22,1879.15,-8.0,187060.0,6582.0,6599.000000,7114.350,7364.500000,17866.383333,17752.263889,11956.331667,6260.0
2022-11-23,1879.15,-8.0,909340.0,6422.0,6501.666667,7015.750,7241.875000,17688.383333,17573.930556,11945.515000,6100.0
2022-11-24,1879.15,-8.0,714993.0,6355.6,6368.333333,6908.150,7138.416667,17505.050000,17396.708333,11937.081667,6100.0
2022-11-25,1879.15,-8.0,1079776.0,6172.0,6313.000000,6817.350,7031.625000,17234.383333,17199.763889,11924.265000,6100.0
2022-11-28,1879.15,-8.0,201478.0,6044.8,6087.333333,6710.925,6927.125000,16932.116667,17006.763889,11910.411667,5664.0
2022-11-29,1879.15,-8.0,77130.0,5911.2,5969.333333,6623.025,6823.791667,16611.983333,16838.875000,11903.151667,5592.0
2022-11-30,1879.15,-8.0,119783.0,5739.6,5799.666667,6543.350,6698.375000,16217.683333,16704.736111,11890.411667,5242.0
2022-12-01,1857.47,NaN,404610.0,5612.0,5693.333333,6459.350,6596.125000,15821.716667,16556.152778,11879.271667,5462.0
2022-12-05,1850.98,NaN,135374.0,5588.4,5673.666667,6389.375,6520.687500,15322.083333,16424.236111,11863.678333,5982.0
2022-12-06,1847.19,NaN,83130.0,5556.0,5574.000000,6297.975,6443.687500,14829.783333,16276.486111,11840.185000,5502.0


## xgboostregressor를 이용한 모델 학습

In [79]:
# {'colsample_bytree': 1, 'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 100, 'reg_alpha': 0, 'reg_lambda': 0, 'subsample': 0.5}


xgb = XGBRegressor(eta = 0.1, max_depth = 3, n_estimators = 100, colsample_bytree = 1, reg_alpha = 0, reg_lambda = 0, subsample = 0.5)
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False, eta=0.1,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.100000001, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0, ...)

In [80]:
y_pred = xgb.predict(X_test)
print("rmse :", round(np.sqrt(mean_squared_error(y_test, y_pred)),3))
print("r2_score :", round(r2_score(y_test, y_pred),3))

rmse : 0.396
r2_score : 0.909


In [77]:
y_pred = xgb.predict(X_train)
print("r2_score :", round(r2_score(y_train, y_pred),3))

r2_score : 0.991


## 12월 예측